In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Week 2.5: Monte Carlo Quant Finance\n",
    "**IIT Bombay CSE 2nd Year | Black-Scholes + Greeks**\n",
    "\n",
    "**S&P500 Call Option Pricing (Dec 2025)**"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "from scipy.stats import norm\n",
    "np.random.seed(42)\n",
    "plt.rcParams['figure.figsize'] = (12, 8)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": ["## **Black-Scholes Monte Carlo Pricing**"]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "class BlackScholesMC:\n",
    "    def __init__(self):\n",
    "        pass\n",
    "    \n",
    "    @staticmethod\n",
    "    def bs_price(S0, K, T, r, sigma, option_type='call'):\n",
    "        '''Exact Black-Scholes formula for validation'''\n",
    "        d1 = (np.log(S0/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))\n",
    "        d2 = d1 - sigma*np.sqrt(T)\n",
    "        \n",
    "        if option_type == 'call':\n",
    "            return S0*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)\n",
    "        else:\n",
    "            return K*np.exp(-r*T)*norm.cdf(-d2) - S0*norm.cdf(-d1)\n",
    "    \n",
    "    def monte_carlo_price(self, S0, K, T, r, sigma, N_paths=100_000):\n",
    "        '''Monte Carlo: Geometric Brownian Motion paths'''\n",
    "        # Terminal stock price: S_T = S0 * exp((r-σ²/2)T + σ√T * Z)\n",
    "        Z = np.random.standard_normal(N_paths)\n",
    "        ST = S0 * np.exp((r - 0.5*sigma**2)*T + sigma*np.sqrt(T)*Z)\n",
    "        \n",
    "        # European call/put payoffs\n",
    "        call_payoff = np.maximum(ST - K, 0)\n",
    "        put_payoff = np.maximum(K - ST, 0)\n",
    "        \n",
    "        # Risk-neutral discounted expectation\n",
    "        call_price = np.exp(-r*T) * np.mean(call_payoff)\n",
    "        put_price = np.exp(-r*T) * np.mean(put_payoff)\n",
    "        \n",
    "        return call_price, put_price, ST\n",
    "    \n",
    "    def greeks_monte(self, S0, K, T, r, sigma, N_paths=500_000):\n",
    "        '''Monte Carlo Greeks (Delta via finite difference)'''\n",
    "        call_p0, _, _ = self.monte_carlo_price(S0, K, T, r, sigma, N_paths)\n",
    "        \n",
    "        # Bump S0 by 1 cent for Delta\n",
    "        eps = 0.01\n",
    "        call_p1, _, _ = self.monte_carlo_price(S0+eps, K, T, r, sigma, N_paths)\n",
    "        delta_mc = (call_p1 - call_p0) / eps\n",
    "        \n",
    "        # Exact Black-Scholes for comparison\n",
    "        exact_call = self.bs_price(S0, K, T, r, sigma)\n",
    "        exact_delta = norm.cdf((np.log(S0/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T)))\n",
    "        \n",
    "        return {\n",
    "            'MC_Call': call_p0, 'Exact_Call': exact_call,\n",
    "            'MC_Delta': delta_mc, 'Exact_Delta': exact_delta\n",
    "        }"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": ["## **S&P500 ATM Call (Dec 2025)**"]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Current market: S&P500 ~5800 (Dec 2025)\n",
    "S0, K, T, r, sigma = 5800, 6000, 0.25, 0.04, 0.15  # 3mo ATM call\n",
    "\n",
    "bs = BlackScholesMC()\n",
    "results = bs.greeks_monte(S0, K, T, r, sigma)\n",
    "\n",
    "print('S&P500 ATM CALL (3 months):')\n",
    "print(f'S0=${S0:,.0f}, Strike=${K:,.0f}, T={T}yr, r={r}, σ={sigma}')\n",
    "print(f'Call Price:  MC=${results[\"MC_Call\"]:.2f} | BS=${results[\"Exact_Call\"]:.2f}')\n",
    "print(f'Delta:       MC={results[\"MC_Delta\"]:.3f} | BS={results[\"Exact_Delta\"]:.3f}')"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": ["## **Convergence Analysis**"]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Monte Carlo convergence\n",
    "N_paths = [1_000, 10_000, 100_000, 500_000, 1_000_000]\n",
    "mc_prices = []\n",
    "mc_stds = []\n",
    "\n",
    "for n in N_paths:\n",
    "    price, _, ST = bs.monte_carlo_price(S0, K, T, r, sigma, n)\n",
    "    mc_prices.append(price)\n",
    "    mc_stds.append(np.std(np.maximum(ST - K, 0)) * np.exp(-r*T) / np.sqrt(n))\n",
    "\n",
    "exact_price = results['Exact_Call']\n",
    "errors = 100 * np.abs(np.array(mc_prices) - exact_price) / exact_price\n",
    "\n",
    "fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))\n",
    "\n",
    "# Price convergence\n",
    "ax1.semilogx(N_paths, mc_prices, 'bo-', linewidth=3, markersize=8, label='MC Price')\n",
    "ax1.axhline(exact_price, color='r', ls='--', linewidth=3, label='Exact BS')\n",
    "ax1.set_xlabel('Paths (log)'); ax1.set_ylabel('Call Price ($)')\n",
    "ax1.set_title('Monte Carlo Convergence')\n",
    "ax1.legend(); ax1.grid(True, alpha=0.3)\n",
    "\n",
    "# Error + Std Dev\n",
    "ax2.semilogx(N_paths, errors, 'ro-', linewidth=3, markersize=8, label='Abs Error %')\n",
    "ax2.semilogx(N_paths, 100*np.array(mc_stds)/exact_price, 'g.-', linewidth=2, label='MC Std %')\n",
    "ax2.set_xlabel('Paths (log)'); ax2.set_ylabel('Error/Std %')\n",
    "ax2.set_title('Error Analysis')\n",
    "ax2.legend(); ax2.grid(True, alpha=0.3)\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.savefig('quant_convergence.png', dpi=300, bbox_inches='tight')\n",
    "plt.show()\n",
    "\n",
    "print('\\nConvergence Results:')\n",
    "for i, n in enumerate(N_paths):\n",
    "    print(f'N={n:7,} | Price=${mc_prices[i]:6.2f} | Error={errors[i]:5.2f}%')"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": ["## **Path Visualization**"]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Visualize 1000 paths\n",
    "_, _, ST = bs.monte_carlo_price(S0, K, T, r, sigma, 1000)\n",
    "\n",
    "plt.figure(figsize=(12, 8))\n",
    "plt.hist(ST, bins=50, density=True, alpha=0.7, color='skyblue', label='S_T Distribution')\n",
    "plt.axvline(K, color='r', lw=3, ls='--', label=f'Strike K=${K:,.0f}')\n",
    "plt.axvline(np.mean(ST), color='g', lw=2, label=f'Mean=${np.mean(ST):.0f}')\n",
    "plt.xlabel('Terminal Stock Price S_T'); plt.ylabel('Density')\n",
    "plt.title('S&P500 Path Distribution (1000 paths)')\n",
    "plt.legend(); plt.grid(True, alpha=0.3)\n",
    "plt.savefig('path_distribution.png', dpi=300, bbox_inches='tight')\n",
    "plt.show()"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  }
 }
}
